In [ ]:
# import pandas as pd

# file_path = "Arabic LLMs Hallucination-OSACT2024-Train.txt"  # Update path if needed
# df = pd.read_csv(file_path, sep="\t")


# df['is_hallucinated'] = df['label'].apply(lambda x: 1 if x in ['F1', 'NF'] else 0)

# output_path = "Corrected_Arabic_LLMs_Hallucination_Train.xlsx"
# df.to_excel(output_path, index=False)

# print("Saved successfully to:", output_path)


In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"
os.environ["WANDB_MODE"] = "offline"

In [ ]:
!pip install -U transformers datasets evaluate


In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments

print("🎉 Transformers imported successfully!")

In [ ]:
import transformers, datasets, evaluate
print(f"Transformers: {transformers.__version__}")  # Should be ~4.xx.x
print(f"Datasets: {datasets.__version__}")          # Should be ~2.xx.x
print(f"Evaluate: {evaluate.__version__}")          # Should be ~0.4.x

In [ ]:
import pandas as pd
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, accuracy_score
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments
from datasets import Dataset, DatasetDict
import evaluate
import random
import os

In [ ]:
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
MODEL_NAME = "aubmindlab/bert-base-arabertv02"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

In [ ]:
train_df = pd.read_excel('Corrected_Arabic_LLMs_Hallucination_Train.xlsx')


In [ ]:
print(train_df.head())

In [ ]:
train_df['is_hallucinated'] = train_df['is_hallucinated'].astype(int)

In [ ]:
train_labels = train_df['is_hallucinated'].tolist()

In [ ]:
train_texts, val_texts, train_labels, val_labels = train_test_split(
    train_df['claim'].tolist(),
    train_df['is_hallucinated'].tolist(),
    test_size=0.2,
    random_state=SEED,
    stratify=train_df['is_hallucinated']
)

In [ ]:
def preprocess_function(examples):
    tokenized = tokenizer(examples['text'], truncation=True, padding='max_length', max_length=128)
    if 'labels' in examples:
        tokenized['labels'] = examples['labels']
    return tokenized

In [ ]:
train_dataset = Dataset.from_dict({'text': train_texts, 'labels': train_labels})
val_dataset = Dataset.from_dict({'text': val_texts, 'labels': val_labels})

In [ ]:
datasets = DatasetDict({
    'train': train_dataset.map(preprocess_function, batched=True),
    'validation': val_dataset.map(preprocess_function, batched=True)
})

In [ ]:
datasets.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'], output_all_columns=True)

# Define model
num_labels = 2
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=num_labels).to(device)

# Metrics computation
metric = evaluate.load("accuracy")

In [ ]:
from sklearn.metrics import f1_score

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {
        'macro_f1': f1_score(labels, predictions, average='macro'),
        'accuracy': accuracy_score(labels, predictions)
    }

In [ ]:
from sklearn.utils.class_weight import compute_class_weight
import torch
import numpy as np

labels = np.array([0, 1])
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(train_labels), y=train_labels)
class_weights = torch.tensor(class_weights, dtype=torch.float).to(device)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model.to(device)
class_weights = class_weights.to(device)


In [ ]:
from collections import Counter
print(Counter(train_labels))

In [ ]:
from transformers import Trainer
import torch.nn as nn

class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        labels = inputs.get("labels")
        outputs = model(**inputs)
        logits = outputs.get("logits")
        loss_fct = nn.CrossEntropyLoss(weight=class_weights)
        loss = loss_fct(logits, labels)
        return (loss, outputs) if return_outputs else loss


In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir='./arabert_classification',
    eval_strategy='epoch',
    save_strategy='epoch',
    learning_rate=1e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=15,
    weight_decay=0.01,
    # no_cuda=True,
    seed=42,
    lr_scheduler_type='linear',
    warmup_ratio= 0.1,
    load_best_model_at_end=True,
    metric_for_best_model='accuracy',
    greater_is_better=True,
    report_to=[],
)


In [ ]:
from transformers import EarlyStoppingCallback

trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=datasets['train'],
    eval_dataset=datasets['validation'],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    # callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
)

In [ ]:
import torch
print(torch.cuda.is_available())


In [ ]:
trainer.train()

In [ ]:
preds = trainer.predict(datasets['validation'])
print(np.unique(preds.predictions.argmax(axis=1), return_counts=True))


In [ ]:
trainer.save_model('./arabert_finetuned')

In [ ]:
test_df = pd.read_csv('Arabic LLMs Hallucination-OSACT2024-Test-NoLabels.txt', sep='\t')
test_texts = test_df['claim'].tolist()
test_ids = test_df['claim_id'].tolist()

test_dataset = Dataset.from_dict({'text': test_texts})
test_dataset = test_dataset.map(preprocess_function, batched=True)
test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask'])

In [ ]:
predictions = trainer.predict(test_dataset)
predicted_labels = np.argmax(predictions.predictions, axis=-1)
predicted_labels_decoded = ['hallucinated' if label == 1 else 'not_hallucinated' for label in predicted_labels]

In [ ]:
output_df = pd.DataFrame({
    'claim_id': test_ids,
    'predicted_label': predicted_labels_decoded
})

In [ ]:
output_df.to_csv('arabert_predictions.csv', index=False)

print("Inference completed. Predictions saved to 'arabert_predictions.csv'.")

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

preds = trainer.predict(datasets['validation'])

y_pred = np.argmax(preds.predictions, axis=-1)
y_true = preds.label_ids

accuracy = accuracy_score(y_true, y_pred)
macro_f1 = f1_score(y_true, y_pred, average='macro')

print(f"✅ Validation Accuracy: {accuracy:.4f}")
print(f"✅ Validation Macro F1: {macro_f1:.4f}")

report = classification_report(y_true, y_pred, target_names=['Not Hallucinated', 'Hallucinated'])
print("\n📊 Classification Report:\n", report)

cm = confusion_matrix(y_true, y_pred)
df_cm = pd.DataFrame(cm, index=['Not Hallucinated', 'Hallucinated'], columns=['Predicted Not', 'Predicted Yes'])

plt.figure(figsize=(6, 4))
sns.heatmap(df_cm, annot=True, fmt='d', cmap='Blues')
plt.title('🧩 Confusion Matrix (Validation Set)')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.show()
